In [2]:
%%configure -f
{
"conf":{
    "spark.sql.extensions":"org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions"
    },
    "jars":[
        "s3://mxtdw-audit-gamma/jars/iceberg-spark3-runtime.jar"
    ]
}

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
24,application_1718585345363_0025,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
22,application_1718585345363_0023,pyspark,idle,Link,Link,None,
24,application_1718585345363_0025,pyspark,idle,Link,Link,None,✔


In [4]:
from pyspark.sql import SparkSession
catalog_name = "glue_catalog"
bucket_name = "mxtdw-audit-gamma"
bucket_prefix = "demo_iceberg/"
database_name = "playground"
warehouse_path = f"s3://{bucket_name}/{bucket_prefix}"
spark = SparkSession.builder \
    .config(f"spark.sql.catalog.{catalog_name}", "org.apache.iceberg.spark.SparkCatalog") \
    .config(f"spark.sql.catalog.{catalog_name}.warehouse", f"{warehouse_path}") \
    .config(f"spark.sql.catalog.{catalog_name}.catalog-impl", "org.apache.iceberg.aws.glue.GlueCatalog") \
    .config(f"spark.sql.catalog.{catalog_name}.io-impl", "org.apache.iceberg.aws.s3.S3FileIO") \
    .config("spark.sql.extensions","org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions") \
    .getOrCreate()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Creating the iceberg table 

In [17]:
%%sql 
drop table if exists glue_catalog.playground.demo2

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

In [6]:
%%sql 
create table glue_catalog.playground.demo4(
    name string,
    phone string,
    email string,
    postalZip long,
    number int,
    country string,
    ingestion_date date
)
using iceberg 
PARTITIONED BY (country,year(ingestion_date))
Location 's3://mxtdw-audit-gamma/demo_iceberg3/'
TBLPROPERTIES ( 
    'write.metadata.metrics.column.postalZip'='full',
    'write.metadata.metrics.column.country'='full',
    'write.metadata.metrics.column.ingestion_date'='full'
);

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

Iceberg Metadata Queries 
Extended list can be found here - https://iceberg.apache.org/docs/nightly/spark-queries/

Getting the s3 files 

In [8]:
%%sql 
select * from glue_catalog.playground.demo4.files;

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

Getting the Snapshots

In [9]:
%%sql 
select * from glue_catalog.playground.demo4.snapshots;

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

Inserting the data 

In [10]:
import pyspark.sql.functions as f
df = spark.read.csv("s3://mxtdw-audit-gamma/demo_data/data.csv",header=True,inferSchema=True)
df = df.withColumn("ingestion_date",f.current_date())

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
df.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------------+--------------+--------------------+---------+------+------------------+--------------+
|           name|         phone|               email|postalZip|number|           country|ingestion_date|
+---------------+--------------+--------------------+---------+------+------------------+--------------+
|   Price Savage|(424) 562-4115|varius.ultrices@g...|   586986|     7|             India|    2024-07-01|
|   Odette Logan|1-562-786-9778|ultricies.ligula@...|     1061|     8|Russian Federation|    2024-07-01|
|Katell Buchanan|(935) 445-9637|magna@protonmail.net|    55526|     9|           Nigeria|    2024-07-01|
|    Leigh Young|1-808-725-8983|ultrices.posuere....|     4906|     7|            Poland|    2024-07-01|
|  Bianca Sexton|1-383-310-3697|risus.at@outlook.net|   115771|     0|           Vietnam|    2024-07-01|
+---------------+--------------+--------------------+---------+------+------------------+--------------+

In [15]:
df.writeTo("glue_catalog.playground.demo4").append()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Querying the Iceberg Table 

In [16]:
%%sql 
select * from glue_catalog.playground.demo4;

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

In [17]:
spark.sql("select * from glue_catalog.playground.demo4.metadata_log_entries").show(truncate=False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------------------+------------------------------------------------------------------------------------------------------+-------------------+----------------+----------------------+
|timestamp              |file                                                                                                  |latest_snapshot_id |latest_schema_id|latest_sequence_number|
+-----------------------+------------------------------------------------------------------------------------------------------+-------------------+----------------+----------------------+
|2024-07-01 15:47:05.973|s3://mxtdw-audit-gamma/demo_iceberg3/metadata/00000-b59e0621-22fe-4121-849f-1375626d8745.metadata.json|null               |null            |null                  |
|2024-07-01 15:53:14.481|s3://mxtdw-audit-gamma/demo_iceberg3/metadata/00001-34b51886-b727-4ba6-bd56-277a00b8c83e.metadata.json|2461266357087007226|0               |1                     |
+-----------------------+------------------------------

In [18]:
spark.sql("select * from glue_catalog.playground.demo4.files").show(truncate=False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+---------------------------------------------------------------------------------------------------------------------------------------------------------+-----------+-------+------------------------+------------+------------------+---------------------------------------------------------------+--------------------------------------------------------+--------------------------------------------------------+----------------+------------------------------------------------------------------------------------------------------------------------------+------------------------------------------------------------------------------------------------------------------------------+------------+-------------+------------+-------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [20]:
spark.sql("select * from glue_catalog.playground.demo4.snapshots").show(truncate=False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------------------+-------------------+---------+---------+------------------------------------------------------------------------------------------------------------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|committed_at           |snapshot_id        |parent_id|operation|manifest_list                                                                                                     |summary                                                                                                                                                                                                                                                                                                      |
+-----------------------+---------

In [21]:
spark.read.format("avro").load("s3://mxtdw-audit-gamma/demo_iceberg3/metadata/snap-2461266357087007226-1-5604a3fe-6e94-415a-8bfe-d0536927aa24.avro").show(truncate=False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------------------------------------------------------------------------------+---------------+-----------------+-------+---------------+-------------------+-------------------+----------------------+-------------------------+------------------------+----------------+-------------------+------------------+------------------------------------------------------------+
|manifest_path                                                                             |manifest_length|partition_spec_id|content|sequence_number|min_sequence_number|added_snapshot_id  |added_data_files_count|existing_data_files_count|deleted_data_files_count|added_rows_count|existing_rows_count|deleted_rows_count|partitions                                                  |
+------------------------------------------------------------------------------------------+---------------+-----------------+-------+---------------+-------------------+-------------------+----------------------+-----------------------

In [22]:
spark.read.format("avro").load("s3://mxtdw-audit-gamma/demo_iceberg3/metadata/5604a3fe-6e94-415a-8bfe-d0536927aa24-m0.avro").show(truncate=False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------+-------------------+---------------+--------------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|status|snapshot_id        |sequence_number|file_sequence_number|data_file                                                                                                                                                                                                    

Running Update command

In [23]:
spark.sql("""
    update glue_catalog.playground.demo4 set postalZip = 75234 
    where number =7;
""")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[]

In [25]:
spark.sql("select * from glue_catalog.playground.demo4").show(truncate=False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------------+--------------+----------------------------------+---------+------+------------------+--------------+
|name           |phone         |email                             |postalZip|number|country           |ingestion_date|
+---------------+--------------+----------------------------------+---------+------+------------------+--------------+
|Katell Buchanan|(935) 445-9637|magna@protonmail.net              |55526    |9     |Nigeria           |2024-07-01    |
|Odette Logan   |1-562-786-9778|ultricies.ligula@icloud.net       |1061     |8     |Russian Federation|2024-07-01    |
|Bianca Sexton  |1-383-310-3697|risus.at@outlook.net              |115771   |0     |Vietnam           |2024-07-01    |
|Price Savage   |(424) 562-4115|varius.ultrices@google.net        |75234    |7     |India             |2024-07-01    |
|Leigh Young    |1-808-725-8983|ultrices.posuere.cubilia@yahoo.com|75234    |7     |Poland            |2024-07-01    |
+---------------+--------------+----------------

In [26]:
spark.sql("select * from glue_catalog.playground.demo4.snapshots order by committed_at desc").show(truncate=False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------------------+-------------------+-------------------+---------+------------------------------------------------------------------------------------------------------------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|committed_at           |snapshot_id        |parent_id          |operation|manifest_list                                                                                                     |summary                                                                                                                                                                                                                                            

In [27]:
spark.sql("select * from glue_catalog.playground.demo4.metadata_log_entries").show(truncate=False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------------------+------------------------------------------------------------------------------------------------------+-------------------+----------------+----------------------+
|timestamp              |file                                                                                                  |latest_snapshot_id |latest_schema_id|latest_sequence_number|
+-----------------------+------------------------------------------------------------------------------------------------------+-------------------+----------------+----------------------+
|2024-07-01 15:47:05.973|s3://mxtdw-audit-gamma/demo_iceberg3/metadata/00000-b59e0621-22fe-4121-849f-1375626d8745.metadata.json|null               |null            |null                  |
|2024-07-01 15:53:14.481|s3://mxtdw-audit-gamma/demo_iceberg3/metadata/00001-34b51886-b727-4ba6-bd56-277a00b8c83e.metadata.json|2461266357087007226|0               |1                     |
|2024-07-01 16:03:13.763|s3://mxtdw-audit-gamma/demo_ic

In [28]:
spark.sql("select * from glue_catalog.playground.demo4.files").show(truncate=False)
#if you take any files from this list , it will only give the updated files which contains the updated data 

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+---------------------------------------------------------------------------------------------------------------------------------------------------------+-----------+-------+------------------------+------------+------------------+---------------------------------------------------------------+--------------------------------------------------------+--------------------------------------------------------+----------------+------------------------------------------------------------------------------------------------------------------------------+------------------------------------------------------------------------------------------------------------------------------+------------+-------------+------------+-------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [31]:
spark.read.parquet("s3://mxtdw-audit-gamma/demo_iceberg3/data/country=India/ingestion_date_year=2024/00000-23-7656577b-d8f1-4969-8956-9c8b3e44c228-00001.parquet").show(truncate=False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------+--------------+--------------------------+---------+------+-------+--------------+
|name        |phone         |email                     |postalZip|number|country|ingestion_date|
+------------+--------------+--------------------------+---------+------+-------+--------------+
|Price Savage|(424) 562-4115|varius.ultrices@google.net|75234    |7     |India  |2024-07-01    |
+------------+--------------+--------------------------+---------+------+-------+--------------+

## Reading Snapshots

In [33]:
spark.sql("select * from glue_catalog.playground.demo4.snapshots").show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+-------------------+-------------------+---------+--------------------+--------------------+
|        committed_at|        snapshot_id|          parent_id|operation|       manifest_list|             summary|
+--------------------+-------------------+-------------------+---------+--------------------+--------------------+
|2024-07-01 15:53:...|2461266357087007226|               null|   append|s3://mxtdw-audit-...|{spark.app.id -> ...|
|2024-07-01 16:03:...|1464511319271994978|2461266357087007226|overwrite|s3://mxtdw-audit-...|{spark.app.id -> ...|
+--------------------+-------------------+-------------------+---------+--------------------+--------------------+

In [35]:
spark.sql("select * from glue_catalog.playground.demo4 version as of 2461266357087007226").show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------------+--------------+--------------------+---------+------+------------------+--------------+
|           name|         phone|               email|postalZip|number|           country|ingestion_date|
+---------------+--------------+--------------------+---------+------+------------------+--------------+
|   Price Savage|(424) 562-4115|varius.ultrices@g...|   586986|     7|             India|    2024-07-01|
|Katell Buchanan|(935) 445-9637|magna@protonmail.net|    55526|     9|           Nigeria|    2024-07-01|
|    Leigh Young|1-808-725-8983|ultrices.posuere....|     4906|     7|            Poland|    2024-07-01|
|   Odette Logan|1-562-786-9778|ultricies.ligula@...|     1061|     8|Russian Federation|    2024-07-01|
|  Bianca Sexton|1-383-310-3697|risus.at@outlook.net|   115771|     0|           Vietnam|    2024-07-01|
+---------------+--------------+--------------------+---------+------+------------------+--------------+

In [58]:
spark.sql("select * from glue_catalog.playground.demo2 version as of 4680088034666527875").show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------------+--------------+--------------------+---------+------+------------------+--------------+
|           name|         phone|               email|postalZip|number|           country|ingestion_date|
+---------------+--------------+--------------------+---------+------+------------------+--------------+
|Katell Buchanan|(935) 445-9637|magna@protonmail.net|    55526|     9|           Nigeria|    2024-06-22|
|   Odette Logan|1-562-786-9778|ultricies.ligula@...|     1061|     8|Russian Federation|    2024-06-22|
|  Bianca Sexton|1-383-310-3697|risus.at@outlook.net|   115771|     0|           Vietnam|    2024-06-22|
|   Price Savage|(424) 562-4115|varius.ultrices@g...|    75234|     7|             India|    2024-06-22|
|    Leigh Young|1-808-725-8983|ultrices.posuere....|    75234|     7|            Poland|    2024-06-22|
+---------------+--------------+--------------------+---------+------+------------------+--------------+

## CDC of tables , which can be used to stream the events 

#### Based on call columns 

In [ ]:
%%sql
CALL glue_catalog.system.create_changelog_view(
  table => 'playground.demo4',
  options => map('start-snapshot-id','2461266357087007226','end-snapshot-id', '1464511319271994978'),
  changelog_view => 'change_load2'
)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

In [37]:
spark.sql("SELECT * FROM change_load2 order by number desc , _change_type desc , country desc").show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------+--------------+--------------------+---------+------+-------+--------------+------------+---------------+-------------------+
|        name|         phone|               email|postalZip|number|country|ingestion_date|_change_type|_change_ordinal|_commit_snapshot_id|
+------------+--------------+--------------------+---------+------+-------+--------------+------------+---------------+-------------------+
| Leigh Young|1-808-725-8983|ultrices.posuere....|    75234|     7| Poland|    2024-07-01|      INSERT|              0|1464511319271994978|
|Price Savage|(424) 562-4115|varius.ultrices@g...|    75234|     7|  India|    2024-07-01|      INSERT|              0|1464511319271994978|
| Leigh Young|1-808-725-8983|ultrices.posuere....|     4906|     7| Poland|    2024-07-01|      DELETE|              0|1464511319271994978|
|Price Savage|(424) 562-4115|varius.ultrices@g...|   586986|     7|  India|    2024-07-01|      DELETE|              0|1464511319271994978|
+------------+------